### Purpose

The intent of this notebook is to serve as a rapid testing ground for new utilities. Any logic written here should migrate to the src/ directory as proper functions.

#### Import and Constants

In [ ]:
from datetime import timedelta
from typing import List

import numpy as np
import pandas as pd
from scipy.ndimage import uniform_filter1d

# Column names
DATE = "date"
WORKOUT_TYPE = "workout_type"
DATA_DURATION = "duration(HH:mm:ss)"  # This is the human-readable version -- it'll be dropped during processing
DURATION = "duration(s)"              # Convert the human-readable durations to seconds for computational simplicity
DISTANCE = "distance(km)"
STEPS = "steps"
ELEVATION = "elevation(m)"
AVG_HEART_RATE = "avg_heart_rate"
MAX_HEART_RATE = "max_heart_rate"
NOTES = "notes"
LOCATION = "location"
EXERCISE = "exercise"
REPS = "reps"
WEIGHT = "weight(lbs)"
RATING = "rating"

#### Load Data

In [ ]:
cardio_workouts = pd.read_csv("../data/cardio_workouts.csv")
weight_training_workouts = pd.read_csv("../data/weight_training_workouts.csv")
travel_days = pd.read_csv("../data/travel_days.csv")
weight_training_sets = pd.read_csv("../data/weight_training_sets.csv")

# Convert dates and times to proper types
for df in [cardio_workouts, weight_training_workouts, travel_days, weight_training_sets]:
    df[DATE] = pd.to_datetime(df[DATE])
    if DATA_DURATION in df:
        df[DATA_DURATION] = pd.to_timedelta(df[DATA_DURATION])
        df[DATA_DURATION] = df[DATA_DURATION].apply(lambda x: int(x.total_seconds()))
        df.rename(columns={DATA_DURATION: DURATION}, inplace=True)
    if NOTES in df:
        df[NOTES] = df[NOTES].fillna("")

# Will be used to pad all datasets to have consistent dates
all_dates = pd.concat([cardio_workouts[DATE], weight_training_workouts[DATE], travel_days[DATE]])

# Filling in a workout type for travel days
travel_days[WORKOUT_TYPE] = "Travel"

#### Transform

In [ ]:
def join_with_comma(items: List[str]):
    """Wrapper function to join a list of strs with commas"""
    return ",".join(items)

def det_workout_type(joined_workout_types: str):
    """Determines the workout type given all of the comma-joined workout types for a given day."""
    result = ""
    for w_type in joined_workout_types.split(","):
        if not result:
            result = w_type
        if result and w_type != result:
            return "Mixed"
    return result

# Initialize the dataframe including all dates spanning the range of the data (rest days are missing in the workout data)
all_workouts = pd.DataFrame()
all_workouts[DATE] = pd.date_range(all_dates.min(), all_dates.max())
all_workouts = all_workouts.set_index(DATE)

# Populate the total daily workout duration
total_durations = pd.concat([
    weight_training_workouts.groupby(DATE)[DURATION].agg(sum),
    cardio_workouts.groupby(DATE)[DURATION].agg(sum),
    travel_days.groupby(DATE)[DURATION].agg(sum),
]).groupby(DATE).agg(sum)
total_durations.index = pd.DatetimeIndex(total_durations.index)
total_durations = total_durations.reindex(all_workouts.index, fill_value=0)
all_workouts[DURATION] = total_durations

# Populate the workout types (there can be more than one per day or rest days, this smooths that out)
workout_types = pd.concat([
    weight_training_workouts.groupby(DATE)[WORKOUT_TYPE].agg(join_with_comma),
    cardio_workouts.groupby(DATE)[WORKOUT_TYPE].agg(join_with_comma),
    travel_days.groupby(DATE)[WORKOUT_TYPE].agg(join_with_comma),
]).groupby(DATE).agg(join_with_comma)
workout_types = workout_types.apply(det_workout_type)
workout_types.index = pd.DatetimeIndex(workout_types.index)
workout_types = workout_types.reindex(all_workouts.index, fill_value="Rest Day")
all_workouts[WORKOUT_TYPE] = workout_types
del workout_types

#### Compute Trends

In [ ]:
# n-day average over a week gives a sense of if I'm keeping above a relatively low baseline of 150 minutes/week
N_DAYS_TO_AVG = 7
n_day_avg_workout_duration = uniform_filter1d(all_workouts[DURATION], size=N_DAYS_TO_AVG)

# TODO Compute other trends:
# * linear regression on weight (first that data needs to be imported of course)
# * logarithmic trendline on resting heartrate (also after importing this data)

#### Build Visuals

In [ ]:
# TODO Build at least these visuals:
# * Weight (plotted w/ linear regression trendline)
# * Resting heartrate (plotted w/ logarithmic trendline)
# * Workout frequency (plotted w/ N day running average)
# * Strength Metrics (PB + 10-rep; format TBD, likely better as two separate graphs or perhaps grouped by workout)
#    - Ideally: drop-down menu to select between various workouts, each plots its PB and 10-rep over time
# * Walking data (max distance, max elevation gain, max duration, pace graph)